In [41]:
# %pip install --upgrade pip setuptools
# %pip install selenium
# %pip install googletrans==4.0.0-rc1

In [42]:
import os
import sys
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import asyncio
import json
import aiohttp
import openpyxl
import textwrap
#from understat import Understat
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd
from googletrans import Translator
from deep_translator import GoogleTranslator

In [43]:
#explanation

In [44]:
url='https://www.rest.co.il/kosher-restaurants/israel/kosher/' #creating url for primary pages
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.64'}
response=requests.get(url,headers=header)
driver = webdriver.Chrome()
page_num=1
resturant_list=[]
infolistArr=[]
info_cnt=0
translator = Translator()
while(page_num<=401):#need 401 pages
    if (page_num%100==0):#every 100 pages the driver closed and go to sleep (for prevent driver error)
        driver.close
        sleep(60)
    if response.status_code != 200:   # if maximum page exceeded, break out of loop
         break
    html_text=requests.get(url,headers=header)
    mainpage=html_text.text
    mainsoup = BeautifulSoup(mainpage,'html.parser') # turn primary page into soup
    rest_page=mainsoup.find_all("div",attrs={"class":"rest-title"})
    
    url_rest_page=[]
    try:
     for div in rest_page:
         href = div.a['href']
         url_rest_page.append("https://www.rest.co.il"+href)
     for url_rest in url_rest_page:
         html_rest_text=requests.get(url_rest,headers=header)
         restpage=html_rest_text.text
         restsoup = BeautifulSoup(restpage,'html.parser')
         rest_name=restsoup.find("div",attrs={"class":"reviews_info_box"})
         rest_name=rest_name.find('h1').text.strip()
         #print(rest_name)
         rest_add=restsoup.find("div", attrs={"class":"h5"}).text.strip()
         try:
             rest_open=restsoup.find("div", attrs={"class":"open_now_dropdown"}).text.strip()
         except AttributeError:
             rest_open='No Info'
         rest_type=restsoup.find("div", attrs={"class":"h6"}).text.strip()
         rest_rank=restsoup.find("div", attrs={"class":"reviews_box"})
         try:
             rest_rank=rest_rank.find("span").text.strip() 
         except AttributeError:
             rest_rank=0
         ratings=restsoup.find_all("div" ,attrs={"class" : "user_progress_widget"})
         name=[]
         rating=[]
         for x in ratings:
             name.append(x.find("span").text.strip())
             rating.append(x.find("strong").text.strip())
         reviews_num=restsoup.find("div",attrs={"class":"raviews_box_item"})
         # print(name)
         try:
             reviews_num=reviews_num.find('a').text.strip().replace("חוות דעת",'') 
         except AttributeError:
             reviews_num=0
         driver.maximize_window()
         driver.get(url_rest)
         try:
             wait = WebDriverWait(driver, 10)
             moreinfo = wait.until(EC.visibility_of_element_located((By.XPATH, "//small[@data-event-id='76' and @data-action='open_tags_popup']")))
             moreinfo.click()
             infolist = driver.find_element(By.CLASS_NAME,'pop-scroll-wrap')
             #sleep(5)
             infolistArr.append(infolist.text.strip())
         except:
             infolistArr.append("None")
         # print("///////////////////////////////////////////")
         
         temp=[]
         for i,x in enumerate(infolistArr):
             temp.append(infolistArr[i].replace('\n',','))
         new_array=[]
         for i,sub_array in enumerate(temp):
             # Create a temporary list to store the separated parts of each string
             temp_list = []
             sub_array=sub_array.split(',')
             new_array=[]            
             for sub in sub_array:
                 new_array.append(f'{sub}')
         # print(new_array)
         restaurant={'rest name':rest_name,'rest add':rest_add,'rest open':rest_open,'rest type':rest_type,'rest rank':rest_rank,'reviews num':reviews_num}

         for i,field in enumerate(name):
             if field is not None:
                 translated = GoogleTranslator(source='auto', target='en').translate(field)
                 if translated is not None:
                     restaurant.update({f'{translated} rank':rating[i]})
                     # print(translated)
             else:
              print("Translation failed for:", field)

         for new in new_array:
             if new is not None:
                 translated = GoogleTranslator(source='auto', target='en').translate(new)
                 if translated is not None:
                     restaurant.update({translated:1})
                     # print(translated)
             else:
              print("Translation failed for:", new)

         #print(restaurant)
         resturant_list.append(restaurant)
         info_cnt=info_cnt+1
    except:
        continue            
    page_num=page_num+1
    url='https://www.rest.co.il/kosher-restaurants/israel/kosher/page-'+ str(page_num) + '/'
    #print(url)   
resturantdf=pd.DataFrame(resturant_list)

In [46]:
resturantdf=pd.DataFrame(resturant_list)
resturantdf.to_csv('TotalData1.csv',encoding='utf-8-sig',index=False)

In [47]:
resturantdf

,rest name,rest add,rest open,rest type,rest rank,reviews num,Food rank,Service rank,atmosphere rank,price rank,...,Open on Saturday,Restaurants on the sea,sports bar,A restaurant with discounts for students,A restaurant in nature,burger club,fast food,Neighborhood bar/pub,Irish pub,Crazy Sunday
0,"רונן בשרים, נתניה","שכטרמן 28, נתניה",א'-ה' 11:00-02:30\nש' חצי שעה אחרי צאת השבת-00:00,מסעדת בשרים,3.5,10,3.5,3.5,3.5,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"עלאש - מסעדת בשרים, מודיעין והסביבה","שד' הרכס 13, מודיעין והסביבה",א'-ה' 12:00-23:00\n\nימי שישי ומוצאי שבת - תיא...,מסעדת בשרים,5,3,5.0,5.0,5.0,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"רג'ינה בתחנה, מתחם התחנה נווה צדק, תל אביב","המרד 3, תל אביב",א'-ה' 17:00-23:30\nו' 11:30-16:00\nש' 19:00-00:00,מסעדת בשרים,4,68,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"סארפסייד - SurfSide, תל אביב","הירקון 115, אבן גבירול, תל אביב",א'-ה' 18:00-23:00,מסעדת בשרים,3,2,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"זוקו - zoko by izidor segal, קריית אונו","שד' קק""ל 5, קריית אונו",א'-ה' 18:30-00:00,מסעדת בשרים,5,3,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6037,"שווארמה ברמה, רמת השרון","שבטי ישראל 106, רמת השרון",No Info,אוכל רחוב,4,1,5.0,4.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,"פלאפל שלמה, ירושלים","מלכי ישראל 5, ירושלים",No Info,אוכל רחוב,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6039,"מאפיית הארץ, חיפה","העליה השניה 33, בת גלים, חיפה, חיפה והקריות",No Info,קונדיטוריה-מאפיה,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6040,"אצל רחל בחצר, אלון מורה","הבשן 142, אלון מורה",No Info,מסעדת בשרים,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
